# Frontend Security Overview

This notebook covers essential frontend security concepts and best practices to protect web applications from common vulnerabilities.

## Topics Covered
- **XSS Prevention** - Cross-Site Scripting defense strategies
- **CSRF Protection** - Cross-Site Request Forgery mitigation
- **Content Security Policy (CSP)** - Browser security policies
- **Secure Cookie Attributes** - HttpOnly, Secure, SameSite
- **HTTPS/TLS** - Transport layer security
- **Input Sanitization** - Validating and cleaning user input
- **Dependency Security** - Managing third-party vulnerabilities
- **Secure Storage Practices** - Safe client-side data storage

## 1. XSS (Cross-Site Scripting) Prevention

XSS attacks inject malicious scripts into trusted websites. There are three main types:
- **Stored XSS**: Malicious script permanently stored on target server
- **Reflected XSS**: Script reflected off web server in error messages, search results
- **DOM-based XSS**: Vulnerability exists in client-side code

### Prevention Strategies
| Strategy | Description |
|----------|-------------|
| Output Encoding | Escape data based on context (HTML, JS, URL, CSS) |
| Content Security Policy | Restrict script sources |
| Use Safe APIs | `textContent` over `innerHTML` |
| Sanitization Libraries | DOMPurify for HTML content |

In [ ]:
# XSS Prevention Examples
from html import escape

# ❌ VULNERABLE: Direct HTML insertion (JavaScript equivalent)
vulnerable_js = '''
// DANGEROUS - Never do this!
element.innerHTML = userInput;  // XSS vulnerability!
document.write(userInput);      // Never use with user data
eval(userInput);                // Code execution vulnerability
'''

# ✅ SECURE: Use textContent or sanitization (JavaScript equivalent)
secure_js = '''
// SAFE - Use textContent
element.textContent = userInput;  // Escapes HTML automatically

// Use DOMPurify for HTML content
import DOMPurify from 'dompurify';
element.innerHTML = DOMPurify.sanitize(userInput);

// React automatically escapes by default
const Component = () => <div>{userInput}</div>;  // Safe!

// Careful with dangerouslySetInnerHTML - sanitize first!
<div dangerouslySetInnerHTML={{__html: DOMPurify.sanitize(html)}} />
'''

# Python server-side escaping
user_input = '<script>alert("XSS")</script>'
safe_output = escape(user_input)
print(f"Original:  {user_input}")
print(f"Escaped:   {safe_output}")

In [ ]:
# CSRF Protection Examples

# ❌ VULNERABLE: No CSRF protection
vulnerable_form = '''
<form action="/transfer" method="POST">
  <input name="amount" value="1000">
  <input name="to" value="attacker">
  <button>Transfer</button>
</form>
<!-- Attacker can submit this from any site! -->
'''

# ✅ SECURE: CSRF token implementation
secure_form = '''
<form action="/transfer" method="POST">
  <input type="hidden" name="csrf_token" value="{{csrf_token}}">
  <input name="amount" value="1000">
  <button>Transfer</button>
</form>
'''

# JavaScript: Include CSRF token in fetch requests
js_csrf = '''
// Get token from meta tag
const csrfToken = document.querySelector('meta[name="csrf-token"]').content;

fetch('/api/transfer', {
  method: 'POST',
  headers: {
    'Content-Type': 'application/json',
    'X-CSRF-Token': csrfToken  // Custom header for AJAX
  },
  credentials: 'same-origin',  // Important: include cookies
  body: JSON.stringify(data)
});
'''

print("CSRF Token Requirements:")
print("• Cryptographically random (min 128 bits)")
print("• Tied to user session")
print("• Validated server-side on every state-changing request")
print("• Regenerate on login to prevent session fixation")

In [ ]:
# Content Security Policy (CSP) Examples

# ❌ VULNERABLE: No CSP or overly permissive
weak_csp = "Content-Security-Policy: default-src *; script-src * 'unsafe-inline' 'unsafe-eval'"

# ✅ SECURE: Strict CSP policies
csp_examples = {
    "Strict Policy": (
        "default-src 'self'; "
        "script-src 'self'; "
        "style-src 'self' 'unsafe-inline'; "  # Often needed for CSS-in-JS
        "img-src 'self' data: https:; "
        "font-src 'self'; "
        "connect-src 'self' https://api.example.com; "
        "frame-ancestors 'none'; "  # Prevents clickjacking
        "base-uri 'self'; "
        "form-action 'self'"
    ),
    "Nonce-based (recommended)": (
        "script-src 'nonce-{RANDOM}' 'strict-dynamic'; "
        "object-src 'none'; "
        "base-uri 'self'"
    ),
}

# CSP Directives Reference
directives = {
    "default-src": "Fallback for other directives",
    "script-src": "Valid JavaScript sources",
    "style-src": "Valid stylesheet sources",
    "img-src": "Valid image sources",
    "connect-src": "URLs for fetch, WebSocket, XHR",
    "frame-ancestors": "Who can embed this page (clickjacking prevention)",
    "report-uri": "Where to send violation reports"
}

print("Key CSP Directives:")
for directive, desc in directives.items():
    print(f"  {directive}: {desc}")

In [ ]:
# Secure Cookie Attributes & HTTPS/TLS

# ❌ VULNERABLE: Insecure cookie
insecure_cookie = "Set-Cookie: session=abc123"

# ✅ SECURE: All security attributes
secure_cookie = (
    "Set-Cookie: session=abc123; "
    "HttpOnly; "         # Prevents JavaScript access (XSS mitigation)
    "Secure; "           # Only sent over HTTPS
    "SameSite=Strict; "  # No cross-site sending (CSRF mitigation)
    "Path=/; "
    "Max-Age=3600"       # 1 hour expiry
)

# Cookie attribute reference
cookie_attrs = {
    "HttpOnly": "Blocks document.cookie access - mitigates XSS token theft",
    "Secure": "Only transmitted over HTTPS connections",
    "SameSite=Strict": "Never sent cross-site (best CSRF protection)",
    "SameSite=Lax": "Sent on top-level navigations only (default in modern browsers)",
    "SameSite=None": "Sent cross-site (requires Secure flag)"
}

# Essential security headers
security_headers = {
    "Strict-Transport-Security": "max-age=31536000; includeSubDomains; preload",
    "X-Content-Type-Options": "nosniff",
    "X-Frame-Options": "DENY",
    "Referrer-Policy": "strict-origin-when-cross-origin",
    "Permissions-Policy": "geolocation=(), camera=(), microphone=()"
}

print("Cookie Security Attributes:")
for attr, desc in cookie_attrs.items():
    print(f"  {attr}: {desc}")

print("\nEssential Security Headers:")
for header, value in security_headers.items():
    print(f"  {header}: {value}")

In [ ]:
# Input Sanitization & Validation
import re
from urllib.parse import urlparse

# ❌ VULNERABLE: No validation - SQL Injection, Path Traversal
def vulnerable_query(user_id):
    return f"SELECT * FROM users WHERE id = {user_id}"  # SQL Injection!

def vulnerable_file(filename):
    return open(f"/uploads/{filename}")  # Path traversal!

# ✅ SECURE: Proper validation and sanitization
def validate_email(email: str) -> bool:
    """Whitelist-based email validation"""
    pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    return bool(re.match(pattern, email)) and len(email) <= 254

def validate_url(url: str, allowed_domains: list) -> bool:
    """Validate URL against allowed domains (prevents SSRF)"""
    try:
        parsed = urlparse(url)
        return (
            parsed.scheme in ('http', 'https') and
            parsed.netloc in allowed_domains
        )
    except:
        return False

def sanitize_filename(filename: str) -> str:
    """Prevent path traversal attacks"""
    # Remove path separators and null bytes
    sanitized = re.sub(r'[/\\\\\0]', '', filename)
    # Remove leading dots (hidden files, parent dir)
    sanitized = sanitized.lstrip('.')
    return sanitized[:255]  # Limit length

# Test examples
print("Validation Examples:")
print(f"  Valid email: {validate_email('user@example.com')}")
print(f"  XSS in email: {validate_email('<script>alert(1)</script>')}")
print(f"  Path traversal sanitized: '{sanitize_filename('../../../etc/passwd')}'")
print(f"  URL validation: {validate_url('https://api.example.com/data', ['api.example.com'])}")

In [ ]:
# Dependency Security & Secure Storage Practices

# Dependency Security Commands
audit_commands = {
    "npm": "npm audit fix --force",
    "yarn": "yarn audit",
    "pnpm": "pnpm audit",
    "pip": "pip-audit",
    "snyk": "snyk test --all-projects",
}

# ❌ VULNERABLE: Storing sensitive data in localStorage
insecure_storage_js = '''
// NEVER store auth tokens in localStorage - accessible via XSS!
localStorage.setItem('authToken', token);
localStorage.setItem('apiKey', 'sk-1234567890');
localStorage.setItem('creditCard', '4111-1111-1111-1111');
'''

# ✅ SECURE: Proper storage patterns
secure_storage_js = '''
// Auth tokens: Use HttpOnly cookies (set by server, not JS)
// For SPAs: Use short-lived access tokens + refresh token rotation

// If token must be in JS (avoid if possible):
let authToken = null;  // Memory only - lost on refresh (that's OK)

// For non-sensitive preferences only:
localStorage.setItem('theme', 'dark');
localStorage.setItem('locale', 'en-US');

// Never store: passwords, tokens, API keys, PII, payment data
'''

# Storage comparison
storage_comparison = {
    "localStorage": {"XSS Vulnerable": "Yes", "Use For": "Non-sensitive preferences"},
    "sessionStorage": {"XSS Vulnerable": "Yes", "Use For": "Temporary non-sensitive data"},
    "HttpOnly Cookie": {"XSS Vulnerable": "No", "Use For": "Auth tokens, sessions"},
    "Memory (variable)": {"XSS Vulnerable": "Partially", "Use For": "Short-lived tokens in SPAs"},
}

print("Dependency Audit Commands:")
for tool, cmd in audit_commands.items():
    print(f"  {tool}: {cmd}")

print("\n⚠️  NEVER store in localStorage/sessionStorage:")
print("  • Auth tokens  • API keys  • PII  • Payment data  • Passwords")

## Security Checklist & Tools

### Pre-Deployment Checklist
- [ ] **CSP headers** configured and tested (use report-only first)
- [ ] **HTTPS enforced** with HSTS header
- [ ] **Cookies** have `HttpOnly`, `Secure`, `SameSite` flags
- [ ] **CSRF tokens** on all state-changing requests
- [ ] **Input validation** on client AND server (server is mandatory)
- [ ] **Output encoding** context-appropriate (HTML, JS, URL, CSS)
- [ ] **Dependencies** audited and updated regularly
- [ ] **No secrets** in client-side code or source control
- [ ] **Sensitive data** not in localStorage/sessionStorage
- [ ] **Security headers**: X-Frame-Options, X-Content-Type-Options, Referrer-Policy

### Security Testing Tools
| Tool | Purpose |
|------|---------|
| **OWASP ZAP** | Automated vulnerability scanner |
| **Burp Suite** | Web security testing proxy |
| **Mozilla Observatory** | HTTP header analysis |
| **CSP Evaluator** | Content Security Policy validation |
| **Snyk / npm audit** | Dependency vulnerability scanning |
| **Lighthouse** | Security audit in Chrome DevTools |

### Key Resources
- [OWASP Cheat Sheet Series](https://cheatsheetseries.owasp.org/)
- [MDN Web Security](https://developer.mozilla.org/en-US/docs/Web/Security)
- [Content Security Policy Reference](https://content-security-policy.com/)